In [1]:
from cal_utils import *
from scipy.optimize import least_squares, minimize

In [ ]:
file_num = 0
folder = ''
def err_iprobe(iprobe_prm_input, solidCube_prm_input, mode):
    return getError_robot(Prm(iprobe_prm_input[0], 
                        iprobe_prm_input[1], 
                        iprobe_prm_input[2]), 
                    solidCube(solidCube_prm_input[0], 
                              solidCube_prm_input[1], 
                              solidCube_prm_input[2], 
                              solidCube_prm_input[3], 
                              solidCube_prm_input[4], 
                              solidCube_prm_input[5]), 
                    mode, file_num, folder)
    
def err_solidcube(solidCube_prm_input, iprobe_prm_input, mode):
    return getError_robot(Prm(iprobe_prm_input[0], 
                        iprobe_prm_input[1], 
                        iprobe_prm_input[2]), 
                    solidCube(solidCube_prm_input[0], 
                              solidCube_prm_input[1], 
                              solidCube_prm_input[2], 
                              solidCube_prm_input[3], 
                              solidCube_prm_input[4], 
                              solidCube_prm_input[5]), 
                    mode, file_num, folder)

In [ ]:
def iterative_optimization(iprobe_prm_input, solidCube_prm_input, 
                           tol=1e-6, max_iter=1000):
    prev_error = np.inf
    for iteration in range(max_iter):
        result_iprobe = minimize(fun=err_iprobe, 
                                 x0=iprobe_prm_input, 
                                 args=(solidCube_prm_input, 'scalar'), 
                                 method='L-BFGS-B', 
                                 bounds=[(53.5, 73.5), 
                                         (67.0, 87.0), 
                                         (-87.0, -67.0)], 
                                 tol=tol,
                                 options={'maxiter': max_iter, 'disp': False})
        iprobe_prm_input = result_iprobe.x
        
        result_solidCube = least_squares(fun=err_solidcube, 
                                         x0=solidCube_prm_input, 
                                         args=(iprobe_prm_input, 'vector'), 
                                         ftol=tol, 
                                         # tr_options={'verbosity': 1, 'max_iter': 2000}, 
                                         verbose=1, 
                                         max_nfev=max_iter)
        solidCube_prm_input = result_solidCube.x
        
        current_error = err_iprobe(iprobe_prm_input, solidCube_prm_input, 'scalar')
        print(f"Iteration {iteration+1}, Error: {current_error:.8f}")
        if abs(prev_error - current_error) < tol:
            print("Converged.")
            break
        prev_error = current_error
    
    return iprobe_prm_input, solidCube_prm_input, current_error
    

In [ ]:
iprobe_prm_input = [63.5, 77.0, -77.0]
solidCube_prm_input = [-2.2266e-03, 6.3928e-02, -5.1277e-05, 
                       5.4682e+00, 7.0481e-04, 4.8222e-04]
# iprobe_prm_input = [66.20844612, 74.07940723, -77.77190349]
# solidCube_prm_input = [5.75713355e-03, 1.65454347e-01, -2.51927934e-03, 
#                        5.47618023e+00, 6.33467621e-02, -4.00017302e-04]

print(f"Before calibration: {err_iprobe(iprobe_prm_input, solidCube_prm_input, 'scalar')}")
final_iprobe_prm, final_solidCube_prm, final_error = iterative_optimization(iprobe_prm_input, solidCube_prm_input)

print("\nFinal optimized parameters:")
print(f"iProbe Parameters: [{final_iprobe_prm[0]}, {final_iprobe_prm[1]}, {final_iprobe_prm[2]}]")
print(f"SolidCube Parameters: [{final_solidCube_prm[0]}, {final_solidCube_prm[1]}, {final_solidCube_prm[2]}, {final_solidCube_prm[3]}, {final_solidCube_prm[4]}, {final_solidCube_prm[5]}]")
print("Final Error:", final_error)